# 챗봇 문답 데이터 감정 분류 모델 구현

예제로 제공되는 데이터는 [github.com/songys/Chatbot_data](https://github.com/songys/Chatbot_data)에서 공개하신 한국어 챗봇 데이터입니다.

In [15]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [16]:
# 데이터 읽어오기
train_file = "ChatbotData.csv"
data = pd.read_csv(train_file, delimiter= ',')
features = data['Q'].tolist()
labels = data['label'].tolist()

In [17]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features] # 위에서 불러온 질문 리스트(features)에서 문장을 하나씩 꺼내와 text_toword_sequence() 함수를 이용해 단어 시퀀스를 만든다.
# 단어 시퀀스란 단어 토큰들의 순차적 리스트를 의미한다. 예를 들어 '아라시는 결성한지 23주년입니다.'는 ['아라시는', '결성한지', '23주년입니다']가 된다.
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus) # 생성된 단어 시퀀스를 말뭉치(corpus) 리스트에 저장한다.
sequences = tokenizer.texts_to_sequences(corpus) # texts_to_sequences() 함수를 이용해 문장 내 모든 단어를 시퀀스 번호로 변환한다. 이를 통해 단어 임베딩 벡터를 만든다.
word_index = tokenizer.word_index

**시퀀스 번호로 만든 벡터는 한 가지 문제가 있는데, 바로 문장의 길이가 제각각이기 때문에 벡터 크기가 모두 다르다. 따라서 시퀀스 번호로 변환된 전체 벡터 크기를 동일하게 맞춰야 한다. MAX_SEQ_LEN 크기만큼 늘리고, MAX_SEQ_LEN보다 작은 벡터는 남는 공간이므로 0으로 채우는 작업을 한다. 이를 패딩(Padding)이라고 한다.**

In [18]:
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post') # pad_sequences () 함수를 통해 시퀀스의 패딩 처리를 할 수 있다.

In [19]:
# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋 : 검증셋 : 테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds= ds.shuffle(len(features))

train_size = int(len(padded_seqs)*0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs)*0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [20]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수

CNN 모델을 케라스 함수형 모델(functional model) 방식으로 구현했다.

문장을 감정 클래스로 분류하는 CNN 모델은 전처리된 입력 데이터를 단어 임베딩 처리하는 영역과 합성곱 필터와 연산을 통해 문장의 특징 정보(특징맵)를 추출하고, 평탄화(flatten)를 하는 영역, 그리고 완전 연결 계층(fully connected layer)을 통해 감정별로 클래스를 분류하는 영역으로 구성된다.

In [24]:
# CNN 모델 정의

# 입력 계층은 keras의 Input()으로 생성, shape의 인자로 입력 노드에 들어올 데이터의 형상(shape)을 지정한다.
# 실제 패딩처리된 시퀀스 벡터의 크기(MAX_SEQ_LEN)로 설정한다.
input_layer = Input(shape=(MAX_SEQ_LEN,)) 

# 앞서 단어별로 패딩 처리된 시퀀스 벡터는 희소벡터다.
# 임베딩 계층은 희소벡터를 입력받아 데이터 손실을 최소화하면서 벡터 차원이 압축되는 밀집 벡터로 변환해준다.
# 단어의 개수(VOCAB_SIZE)와 임베딩 결과로 나올 밀집 벡터의 크기(EMB_SIZE), 입력되는 시퀀스 벡터의 크기(MAX_SEQ_LEN)를 Embedding()의 인자로 사용해 임베딩 계층을 생성한다.
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length = MAX_SEQ_LEN)(input_layer) 
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer) # Overfitting(과적합)을 막기 위해 50% 확률로 Dropout()

# 임베딩 계층을 통해 전달된 임베딩 벡터에서 특징 추출을 하는 영역 구현
# Conv1D()을 이용해 크기가 3, 4, 5인 합성곱 filter를 128개씩 사용한 합성곱 계층을 3개 생성, 이는 3, 4, 5-gram 언어 모델의 개념고 ㅏ비슷하다.
conv1=Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation = tf.nn.relu)(dropout_emb)
pool1 =GlobalMaxPool1D()(conv1) # 임베딩 벡터를 합성곱 계층의 입력으로 받아 GlobalMaxPool1D()를 이용해 최대 풀링 연산을 수행한다.

conv2 = Conv1D(
    filters = 128,
    kernel_size=4,
    padding='valid',
    activation= tf.nn.relu)(dropout_emb)
pool2= GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [25]:
# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1,pool2,pool3]) #완전 연결 계층에 전달될 수 있도록 concatenate()를 이용해 각각 병렬로 처리된 합성곱 계층의 특징맵 결과를 하나로 묶어준다.

# Dense()를 이용해 128개의 출력노드를 가지고, relu 활성화 함수를 사용하는 Dense 계층을 생성한다.
# 이 Dense 계층은 이전 계층에서 합성곱 연산과 맥스 풀링으로 나온 3개의 특징맵 데이터를 입력으로 받는다.
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)

# 챗봇 데이터 문장에서 3가지 클래스로 감정 분류해야 하기 때문에 출력노드가 3개인 Dense()를 생성한다.
# 이 때 최종 단계이기 때문에 활성화 함수를 사용하지 않는다.
# 이번 계층에서 결과로 나온 값을 logits, score(점수)라고 부른다.
logits= Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation= tf.nn.softmax)(logits)

In [26]:
# 모델 생성
model= Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
414/414 [==============================] - 16s 35ms/step - loss: 0.8960 - accuracy: 0.5632 - val_loss: 0.5428 - val_accuracy: 0.7944
Epoch 2/5
414/414 [==============================] - 13s 32ms/step - loss: 0.5257 - accuracy: 0.8001 - val_loss: 0.2693 - val_accuracy: 0.9192
Epoch 3/5
414/414 [==============================] - 13s 32ms/step - loss: 0.3125 - accuracy: 0.8931 - val_loss: 0.1606 - val_accuracy: 0.9509
Epoch 4/5
414/414 [==============================] - 13s 32ms/step - loss: 0.1993 - accuracy: 0.9386 - val_loss: 0.1062 - val_accuracy: 0.9700
Epoch 5/5
414/414 [==============================] - 13s 32ms/step - loss: 0.1393 - accuracy: 0.9578 - val_loss: 0.0617 - val_accuracy: 0.9805


In [28]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy*100))
print('loss: %f' % (loss))

60/60 [==============================] - 0s 4ms/step - loss: 0.0544 - accuracy: 0.9831
Accuracy: 98.307955
loss: 0.054410


In [29]:
# 모델 저장
model.save('cnn_model.h5')

#챗봇 문답 데이터 감정 분류 모델 사용

In [31]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [32]:
# 데이터 읽어오기
train_file = "ChatbotData.csv"
data = pd.read_csv(train_file, delimiter=',') # read_csv() 함수를 이용해 파일을 읽어와 
features= data['Q'].tolist() # label(감정)을 분류할 Q (질문) 데이터를 features 리스트에 저장한다.
labels = data['label'].tolist() # labels 리스트는 CNN 모델이 예측한 분류 결과와 실제 분류값을 비교하기 위한 목적

In [33]:
# 단어 인덱스 시퀀스 벡터

# 위에서 불러온 질문 리스트(features)에서 한 문장씩 꺼내와 text_to_word_sequence() 함수를 이용해 단어 시퀀스를 만든 후 말뭉치(corpus) 리스트에 저장한다.
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences= tokenizer.texts_to_sequences(corpus) # 모든 단어를 시퀀스 번호로 변환한다.

In [34]:
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen = MAX_SEQ_LEN, padding='post') # pad_ sequences() 함수를 통해 단어 시퀀스 벡터 크기를 맞춰 패딩 처리를 한다.

In [35]:
# 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds=ds.take(2000).batch(20)

In [37]:
# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5') # load_model() 함수르ㅡㄹ 이용해 모델 파일을 불러온다.
model.summary() # summary() 함수를 호출하고, 테스트셋 데이털르 이용해 모델 성능을 평가한다.
model.evaluate(test_ds, verbose=2) 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 15, 128)      1715072     ['input_5[0][0]']                
                                                                                                  
 dropout_4 (Dropout)            (None, 15, 128)      0           ['embedding_4[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 13, 128)      49280       ['dropout_4[0][0]']              
                                                                                              

[0.059533897787332535, 0.9829999804496765]

In [38]:
# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212]) # 2는 "사랑(Label:2)"이다.

단어 시퀀스 :  ['썸', '타는', '여자가', '남사친', '만나러', '간다는데', '뭐라', '해']
단어 인덱스 시퀀스 :  [   13    61   127  4320  1333 12162   856    31     0     0     0     0
     0     0     0]
문장 분류(정답) :  2


In [39]:
#테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks]) # predict() 함수는 입력 데이터에 대해 각 클래스별로 예측한 점수를 반환한다.
predict_class = tf.math.argmax(predict,axis=1) #argmax() 함수를 이용해 분류 클래스들 중 예측 점수가 가장 큰 클래스 번호를 계산한다. 즉, 10212번째 문장이 어떤 감정 클래스에 포함되어 있는지 판단한다.
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

감정 예측 점수 :  [[1.1718092e-06 9.2785473e-07 9.9999785e-01]]
감정 예측 클래스 :  [2]
